In [ ]:
# Train a model to predict toxicity.
import numpy as np
import deepchem as dc

In [2]:
# Load the data.
tox21_tasks, tox21_datasets, transformers = dc.molnet.load_tox21()
train_dataset, valid_dataset, test_dataset = tox21_datasets

In [3]:
train_dataset.X.shape, train_dataset.y.shape

((6264, 1024), (6264, 12))

In [ ]:
# Build and train the model.
model = dc.models.MultitaskClassifier(n_tasks=12, n_features=1024, layer_sizes=[1000])
model.fit(train_dataset, nb_epoch=10)

In [ ]:
# Evaluate it on the training and test sets.
metric = dc.metrics.Metric(dc.metrics.roc_auc_score, np.mean)
train_scores = model.evaluate(train_dataset, [metric], transformers)
test_scores = model.evaluate(test_dataset, [metric], transformers)
print(train_scores)
print(test_scores)

# MNIST

In [ ]:
# Train a model to recognize handwritten digits.
import os
import deepchem as dc
import tensorflow as tf
import tensorflow.keras.layers as layers

In [8]:
# Load the dataset and convert the labels to one-hot.
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
y_train = tf.one_hot(y_train, 10).numpy()
y_test = tf.one_hot(y_test, 10).numpy()
train_dataset = dc.data.NumpyDataset(x_train, y_train)
test_dataset = dc.data.NumpyDataset(x_test, y_test)

In [9]:
# Build the model.
features = tf.keras.Input(shape=(28, 28, 1))
conv2d_1 = layers.Conv2D(filters=32, kernel_size=5, activation=tf.nn.relu)(features)
conv2d_2 = layers.Conv2D(filters=64, kernel_size=5, activation=tf.nn.relu)(conv2d_1)
flatten = layers.Flatten()(conv2d_2)
dense1 = layers.Dense(units=1024, activation=tf.nn.relu)(flatten)
dense2 = layers.Dense(units=10, activation=None)(dense1)
output = layers.Activation(tf.math.softmax)(dense2)
keras_model = tf.keras.Model(inputs=features, outputs=[output, dense2])

model = dc.models.KerasModel(
    keras_model,
    loss=dc.models.losses.SoftmaxCrossEntropy(),
    output_types=['prediction', 'loss'],
    model_dir='mnist')

In [ ]:
# Train the model.
os.environ["TF_USE_LEGACY_KERAS"] = "True"
model.fit(train_dataset, nb_epoch=10)

In [ ]:
# Evaluate it on the training and test sets.
metric = dc.metrics.Metric(dc.metrics.accuracy_score)
train_scores = model.evaluate(train_dataset, [metric])
test_scores = model.evaluate(test_dataset, [metric])
print(train_scores)
print(test_scores)